# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + λ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, λ)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os
os.listdir("test_images/")

run your solution on all test_images and make copies into the test_images directory).

In [ ]:
# TODO: Build your pipeline that will draw lane lines on the test_images
# then save them to the test_images directory.
plt.close('all')
for file_name in os.listdir("test_images/"):
    image = mpimg.imread("test_images/" + file_name)
    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)    

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 9
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 40
    high_threshold = 120
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 325), (575, 325), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)

    left_x = []
    left_y = []
    left_x_bar = 0
    left_y_bar = 0
    left_m = 0
    left_b = 0
    
    right_x = []
    right_y = []
    right_x_bar = 0
    right_y_bar = 0
    right_m = 0
    right_b = 0
    
    left_x_x_bar = []
    left_y_y_bar = []
    
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            #print(line)
            #print((y2-y1)/(x2-x1))
            
            if(x1 > 500 or x2 > 500):
                right_x.append(x1)
                right_x.append(x2)
                right_y.append(y1)
                right_y.append(y2)
            elif(x1 < 500 or x2 < 500):
                left_x.append(x1)
                left_x.append(x2)
                left_y.append(y1)
                left_y.append(y2)

    for value in left_x:
        left_x_bar += value
    left_x_bar = left_x_bar/len(left_x)
    
    for value in left_y:
        left_y_bar += value
    left_y_bar = left_y_bar/len(left_y)
    
    for value in right_x:
        right_x_bar += value
    right_x_bar = right_x_bar/len(right_x)
    
    for value in right_y:
        right_y_bar += value
    right_y_bar = right_y_bar/len(right_y)
    
    sum_top = 0
    sum_bottom = 0
    
    m_left = 0
    m_right = 0
    
    for index in range(len(left_x)):
        sum_top += ((left_x[index]-left_x_bar)*(left_y[index]-left_y_bar))
        sum_bottom += ((left_x[index] - left_x_bar)**2)
    
    m_left = sum_top/sum_bottom

    sum_top = 0
    sum_bottom = 0
    
    for index in range(len(right_x)):
        sum_top += ((right_x[index]-right_x_bar)*(right_y[index]-right_y_bar))
        sum_bottom += ((right_x[index] - right_x_bar)**2)
    
    m_right = sum_top/sum_bottom
    
    b_left = (left_y_bar) - (m_left * left_x_bar)
    b_right = (right_y_bar) - (m_right * right_x_bar)
    
    cv2.line(line_image, (int((325-b_left)/m_left), 325), (int((540-b_left)/m_left), 540), (0,255,0), 10)
    cv2.line(line_image, (int((325-b_right)/m_right), 325), (int((540-b_right)/m_right), 540), (0,255,0), 10)

    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, .8, line_image, 1, 0) 
    plt.figure()
    plt.imshow(lines_edges)
    lines_edges = cv2.cvtColor(lines_edges, cv2.COLOR_BGR2RGB)
    cv2.imwrite("test_images2/lines_" + file_name , lines_edges)
    


## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)

    gray = cv2.cvtColor(image,cv2.COLOR_RGB2GRAY)
    

    # Define a kernel size and apply Gaussian smoothing
    kernel_size = 9
    blur_gray = cv2.GaussianBlur(gray,(kernel_size, kernel_size),0)

    # Define our parameters for Canny and apply
    low_threshold = 40
    high_threshold = 120
    edges = cv2.Canny(blur_gray, low_threshold, high_threshold)

    # Next we'll create a masked edges image using cv2.fillPoly()
    mask = np.zeros_like(edges)   
    ignore_mask_color = 255   

    # This time we are defining a four sided polygon to mask
    imshape = image.shape
    vertices = np.array([[(0,imshape[0]),(450, 325), (575, 325), (imshape[1],imshape[0])]], dtype=np.int32)
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    masked_edges = cv2.bitwise_and(edges, mask)

    # Define the Hough transform parameters
    # Make a blank the same size as our image to draw on
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 8     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 10 #minimum number of pixels making up a line
    max_line_gap = 10    # maximum gap in pixels between connectable line segments
    line_image = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    # Output "lines" is an array containing endpoints of detected line segments
    lines = cv2.HoughLinesP(masked_edges, rho, theta, threshold, np.array([]),
                            min_line_length, max_line_gap)

    left_x = []
    left_y = []
    left_x_bar = 0
    left_y_bar = 0
    left_m = 0
    left_b = 0
    
    right_x = []
    right_y = []
    right_x_bar = 0
    right_y_bar = 0
    right_m = 0
    right_b = 0
    
    left_x_x_bar = []
    left_y_y_bar = []
    
    # Iterate over the output "lines" and draw lines on a blank image
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            if(x1 > 500 or x2 > 500):
                slope = ((y2-y1)/(x2-x1))
                if(slope > 0.4 and slope < 0.8):
                    right_x.append(x1)
                    right_x.append(x2)
                    right_y.append(y1)
                    right_y.append(y2)
            elif(x1 < 500 or x2 < 500):
                slope = ((y2-y1)/(x2-x1))
                if(slope > -1 and slope < -0.5):
                    left_x.append(x1)
                    left_x.append(x2)
                    left_y.append(y1)
                    left_y.append(y2)

    for value in left_x:
        left_x_bar += value
    left_x_bar = left_x_bar/len(left_x)
    
    for value in left_y:
        left_y_bar += value
    left_y_bar = left_y_bar/len(left_y)
    
    for value in right_x:
        right_x_bar += value
    right_x_bar = right_x_bar/len(right_x)
    
    for value in right_y:
        right_y_bar += value
    right_y_bar = right_y_bar/len(right_y)
    
    sum_top = 0
    sum_bottom = 0
    
    m_left = 0
    m_right = 0
    
    for index in range(len(left_x)):
        sum_top += ((left_x[index]-left_x_bar)*(left_y[index]-left_y_bar))
        sum_bottom += ((left_x[index] - left_x_bar)**2)
    
    m_left = sum_top/sum_bottom
    
    sum_top = 0
    sum_bottom = 0
    
    for index in range(len(right_x)):
        sum_top += ((right_x[index]-right_x_bar)*(right_y[index]-right_y_bar))
        sum_bottom += ((right_x[index] - right_x_bar)**2)
    
    m_right = sum_top/sum_bottom
    
    b_left = (left_y_bar) - (m_left * left_x_bar)
    b_right = (right_y_bar) - (m_right * right_x_bar)
    
    cv2.line(line_image, (int((325-b_left)/m_left), 325), (int((540-b_left)/m_left), 540), (0,255,0), 10)
    cv2.line(line_image, (int((325-b_right)/m_right), 325), (int((540-b_right)/m_right), 540), (0,255,0), 10)
    


    # Draw the lines on the edge image
    lines_edges = cv2.addWeighted(image, 0.9, line_image, 1, 0) 
    result = lines_edges
    
    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

Right now my code relies on individual frames for line detection, using multiple frames for line reference would likely make it more accurate when there are erroneous detections. It may be possible that using single color channels (R, G, or B) could more accurately depict the lines on the road, and thus make them easier for detection, or even using different color spaces (HSV, HLS, etc.)

My current method of detecting lane lines takes all of the points, separates them based on the side of the road, and eliminates lines with an inappropriate slope. This method works well when the lines are straight, clearly marked against a dark background, and the image contains no other (or few) additional objects or markings in the masked section of the image. This means that in a situation like the extra video, when there is a transition from asphalt to concrete the current pipeline doesnt know how to handle the extra information. There is also an issue with the lower section of the image containing the hood of the car. This pipeline is also only accurate with straight lines, curved lines will throw off the current method of fitting a straight best fit line do the data points.


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))